In [6]:
import os
import numpy as np
import matplotlib.pyplot as plt

os.makedirs('./results', exist_ok=True)

with open('./brown_100.txt', 'r') as file_:
    corpus = file_.read()

def write_freqs(list_: list, filename: str) -> None:
    with open(f'./results/{filename}.txt', 'w') as file_:
        file_.write('\n'.join([' '.join(x[0]) + ' | ' + str(x[1]) for x in list_]))

## N-Gram Language Modeling
In the second assignment, you will implement an **n-gram language model** that processes input text to generate, analyze, and calculate the probabilities of sequences of `n` words (n-grams). This model will tokenize text, create n-grams, and compute their frequencies and probabilities, returning the most frequent n-grams in the text.

1. **Tokenization**: Convert the input text into a list of words (tokens).
2. **N-gram Generation**: Create sequences of `n` tokens from the text, treating each sentence independently with the inclusion of start (`<s>`) and end (`</s>`) markers.
3. **Frequency Counting**: Count how many times each n-gram appears in the text.
4. **Probability Calculation**: Compute the probabilities of each n-gram based on its frequency and add alpha smoothing to improve generalization of items.
5. **Most Frequent N-grams**: Return the most frequent n-grams along with their probabilities, helping to identify common patterns in the text.

In [16]:
from collections import Counter, defaultdict
import re


class NGramModel:
    def __init__(self, text, n, alpha=0.0):
        """
        Initialize the NGramModel with text and the value of n.
        """
        self.text = text
        self.n = n
        self.alpha = alpha  # Alpha value for additive smoothing
        self.ngrams = defaultdict(int)
        self.probabilities = {}
        self.vocab = set()
        self.total_ngrams = 0


    def tokenize(self) -> list:
        """
        Tokenize the text into words.
        Add start and end tokens for sentence boundaries.
        Supports multiple sentences.
        """
        # Split text into sentences using punctuation (., !, ?)
        sentences = re.split(r'(?<=[.!?])\s+', self.text.strip())
        remove_punctuation = re.compile(r'[^\w\s]')
        sentences = [remove_punctuation.sub('', sentence) for sentence in sentences]

        tokens = []
        for sentence in sentences:
            if sentence:  # skip empty sentences
                words = sentence.split()
                tokens.extend(['<s>'] * (self.n - 1) + words + ['</s>'])

        return tokens

    def generate_ngrams(self, tokens: list) -> dict:
        """
        Generate n-grams from the list of tokens.
        Treat each sentence independently (simplified for single sentence input).
        """
        if self.n>= 1:
            self.ngrams.clear()  # Clear previous n-grams 
            for i in range(len(tokens) - self.n + 1):
                ngram = tuple(tokens[i:i + self.n])
                self.ngrams[ngram] += 1
                self.vocab.update(ngram)
        else:
            raise ValueError("n must be greater than 1 for ngrams.")
        return self.ngrams

    def count_frequencies(self) -> None:
        """
        Count the frequencies of each n-gram.
        """
        self.total_ngrams = sum(self.ngrams.values())
        return self.total_ngrams

    def calculate_probabilities(self) -> None:
        """
        Calculate probabilities of each n-gram based on its frequency.
        Additive smoothing applied.
        """
        if not self.ngrams:
            raise ValueError("Frequency of N-grams is not yet calculated.")
        
        V = len(self.vocab)
        for ngram, count in self.ngrams.items():
            self.probabilities[ngram] = (count + self.alpha) / (self.total_ngrams + self.alpha * V)
        return self.probabilities

    def most_frequent_ngrams(self, top_n: int = 10) -> list:
        """
        Return the most frequent n-grams and their probabilities.
        """
        sorted_ngrams = sorted(self.ngrams.items(), key=lambda x: x[1], reverse=True)[:top_n]
        return [(ngram, self.probabilities[ngram]) for ngram, _ in sorted_ngrams]


# Example usage
text = "This is a simple example to demonstrate how n-grams work in this assignment."
n = 2
model = NGramModel(text, n)

tokens = model.tokenize()
model.generate_ngrams(tokens)
model.count_frequencies()
model.calculate_probabilities()

# Print the most frequent n-grams
print(model.most_frequent_ngrams(10))


[(('<s>', 'This'), 0.07142857142857142), (('This', 'is'), 0.07142857142857142), (('is', 'a'), 0.07142857142857142), (('a', 'simple'), 0.07142857142857142), (('simple', 'example'), 0.07142857142857142), (('example', 'to'), 0.07142857142857142), (('to', 'demonstrate'), 0.07142857142857142), (('demonstrate', 'how'), 0.07142857142857142), (('how', 'ngrams'), 0.07142857142857142), (('ngrams', 'work'), 0.07142857142857142)]


### Unigrams

In this section, we apply the `NGramModel` to our corpus. The model can be configured to generate unigrams, bigrams, or trigrams by adjusting the value of `n`. After tokenizing the text and generating the n-grams, the model counts the occurrences of each n-gram and calculates their probabilities. The most frequent n-grams are then written to an output file.

By adjusting the n-gram size, we can analyze different levels of word dependencies in the text, providing insights into common word sequences and patterns.  

The code below will generate unigrams when the NGramModel is implemented correctly

In [17]:
n = 1  # You can change this to 1, 2, or 3 for uni, bi, or trigrams
model = NGramModel(corpus, n)

tokens = model.tokenize()
ngrams = model.generate_ngrams(tokens)
model.count_frequencies()
model.calculate_probabilities()

write_freqs(model.most_frequent_ngrams(10), 'unigrams')

### Bigrams
Bigrams allow us to explore word pairs and their relationships, providing insight into common word combinations and phrase structures within the corpus.  
The code below will generate bigrams when the NGramModel is implemented correctly.

In [18]:
n = 2  # You can change this to 1, 2, or 3 for uni, bi, or trigrams
model = NGramModel(corpus, n)

tokens = model.tokenize()
ngrams = model.generate_ngrams(tokens)
model.count_frequencies()
model.calculate_probabilities()

write_freqs(model.most_frequent_ngrams(10), 'bigrams')

### Trigrams
Trigrams capture three-word sequences, providing a deeper understanding of longer word dependencies and commonly occurring phrases within the text.  
The code below will generate trigrams when the NGramModel is implemented correctly

In [19]:
n = 3  # You can change this to 1, 2, or 3 for uni, bi, or trigrams
model = NGramModel(corpus, n)

tokens = model.tokenize()
ngrams = model.generate_ngrams(tokens)
model.count_frequencies()
model.calculate_probabilities()

write_freqs(model.most_frequent_ngrams(10), 'trigrams')

### Smoothing
Below, we generate **bigrams** with **smoothing** applied by setting `n = 2` and including a smoothing parameter `alpha = 1.0`. Smoothing is useful in language models to handle unseen n-grams by assigning a small probability to them. The model tokenizes the text, generates bigrams, counts their frequencies, and calculates smoothed probabilities. The most frequent bigrams are then written to an output file.





In [20]:
n = 2  # You can change this to 1, 2, or 3 for uni, bi, or trigrams
model = NGramModel(corpus, n, alpha=1.0)

tokens = model.tokenize()
ngrams = model.generate_ngrams(tokens)
model.count_frequencies()
model.calculate_probabilities()

write_freqs(model.most_frequent_ngrams(10), 'bigrams_smoothed')

In [21]:
n = 2  # You can change this to 1, 2, or 3 for uni, bi, or trigrams
model = NGramModel(corpus, n, alpha=0.5)

tokens = model.tokenize()
ngrams = model.generate_ngrams(tokens)
model.count_frequencies()
model.calculate_probabilities()

write_freqs(model.most_frequent_ngrams(10), 'bigrams_smoothed_0.5_alpha')

In [22]:
n = 2  # You can change this to 1, 2, or 3 for uni, bi, or trigrams
model = NGramModel(corpus, n, alpha=2)

tokens = model.tokenize()
ngrams = model.generate_ngrams(tokens)
model.count_frequencies()
model.calculate_probabilities()

write_freqs(model.most_frequent_ngrams(10), 'bigrams_smoothed_2_alpha')

### Generating Text
In this example, we generate text using the **bigrams** model by providing a starting **prompt** (e.g., "the jury"). The `generate_text` function creates a sequence of words based on the trained bigram model, starting from the provided prompt and continuing based on the probabilities of the next words.

After tokenizing the corpus, generating bigrams, and calculating their frequencies and probabilities, the model uses these learned probabilities to generate text. The generated output is then written to a file.

Text generation with n-grams helps illustrate how language models can predict word sequences, allowing us to create new sentences that follow similar patterns found in the original text.

In [42]:
import random 
def generate_text(model: NGramModel, n: int, prompt: str, max_tokens: int =15) -> str:
    # Generate text based on the n-gram model and a given prompt.
    if n != model.n:
        raise ValueError(f"Model is built for n={model.n}, but n={n} was provided.")
    if not prompt:
        raise ValueError("Prompt is empty. Please provide a valid prompt.")
    generated_tokens = prompt.split()
    if len(generated_tokens) < n - 1:
        raise ValueError(f"Prompt must have at least {n - 1} tokens for an n-gram model with n={n}")
    for i in range(max_tokens):
        # Get the last n-1 tokens to form the context for prediction
        prev_tokens = tuple(generated_tokens[-(n - 1):])
        
        # Get the probabilities of the next token given the previous n-1 tokens
        next_token_probs = {
            ngram: prob for ngram, prob in model.probabilities.items()
            if ngram[:-1] == prev_tokens
        }

        if not next_token_probs:
            break

        candidates = list(next_token_probs.keys())# List of n-gram tuples
        weights = list(next_token_probs.values()) # Corresponding probabilities

        next_ngram = random.choices(candidates, weights=weights, k=1)[0]# k refers to the number of words to be selected from the candidates list
        next_token = next_ngram[-1]  # Extract the predicted tokens

        if next_token == '</s>':
            break
        generated_tokens.append(next_token)
    
    return ' '.join(generated_tokens)
   
tokens = model.tokenize()
ngrams = model.generate_ngrams(tokens)
model.count_frequencies()
model.calculate_probabilities()
generated_text = generate_text(model, 2, 'the jury')
with open(f'./results/generated_bigrams.txt', 'w') as file_:
    file_.write(generated_text)

### Perplexity:

In [46]:
# To check the perplexity of the model, we can use the following function:
def calculate_perplexity(model: NGramModel, test_text: str) -> float:
    """
    Calculate the perplexity of the model on a given test text.
    """
    tokens = model.tokenize()
    ngrams = model.generate_ngrams(tokens)
    model.count_frequencies()
    model.calculate_probabilities()

    log_prob_sum = 0.0
    for i in range(len(tokens) - model.n + 1):
        ngram = tuple(tokens[i:i + model.n])
        if ngram in model.probabilities:
            log_prob_sum += np.log(model.probabilities[ngram])
        else:
            log_prob_sum += np.log(model.alpha / (model.total_ngrams + model.alpha * len(model.vocab)))

    perplexity = np.exp(-log_prob_sum / (len(tokens) - model.n + 1))
    return perplexity

test_text_1 = " we use this function to check the perplexity of the model."
test_text_2 = "This is a test sentence to evaluate the perplexity of the n-gram model."
test_text_3 = "The fox jumps over the fruits in the farm."
model = NGramModel(test_text_1, n=2, alpha=1.0)
perplexity_1 = calculate_perplexity(model, test_text_1)
model = NGramModel(test_text_2, n=2, alpha=1.0)
perplexity_2 = calculate_perplexity(model, test_text_2)
model = NGramModel(test_text_3, n=2, alpha=1.0)
perplexity_3 = calculate_perplexity(model, test_text_3)

print(f"Perplexity of the model on the test text: \ntext_1: {perplexity_1}, \ntext_2: {perplexity_2}, \ntext_3 {perplexity_3}")

Perplexity of the model on the test text: 
text_1: 12.000000000000005, 
text_2: 13.999999999999996, 
text_3 10.000000000000002


## Word Vectors
Based on the Stanford course: https://web.stanford.edu/class/cs224n/

### Count based W2V
In this section, we use the previously trained **n-gram model** to calculate a **co-occurrence matrix**. A co-occurrence matrix shows how often words appear together within a specific context (in this case, bigrams), providing insights into word relationships in the corpus.

Steps:
1. **Tokenization and N-gram Generation**: We first tokenize the text and generate bigrams using the `NGramModel`.
2. **Vocabulary and Indexing**: The vocabulary is created from the unique tokens, and a word-to-index mapping is established.
3. **Co-occurrence Matrix**: Using the generated n-grams, we compute the co-occurrence matrix, where each entry represents how often two words occur together within the bigrams.

The co-occurrence matrix provides a numerical representation of word relationships, which is useful for tasks like word embeddings, semantic analysis, and understanding word associations.

In [43]:
# Since we already have a working ngram model, for the next part we can use the ngrams to calculate the co-occurences.
model = NGramModel(corpus, 2)
tokens = model.tokenize()
ngrams = model.generate_ngrams(tokens)

def create_co_matrix(ngrams: dict, vocab_size: int, word_to_index: dict) -> np.ndarray:
    """
    Create a co-occurrence matrix from bigrams.
    ngrams: dictionary of n-grams with their frequencies
    vocab_size: number of unique words in vocabulary
    word_to_index: mapping from word to its index in the matrix
    """
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=int)

    for (w1, w2), count in ngrams.items():
        i, j = word_to_index[w1], word_to_index[w2]
        co_matrix[i][j] += count

    return co_matrix

# Build vocabulary and mappings
vocab = list(set(tokens))
vocab_size = len(vocab)
word_to_index = {word: idx for idx, word in enumerate(vocab)}
index_to_word = {idx: word for word, idx in word_to_index.items()}

# Generate co-occurrence matrix
co_matrix = create_co_matrix(ngrams, vocab_size, word_to_index)

# Print top co-occurring word pairs
print("Top co-occurring word pairs:")
co_occurrences = []
for i in range(vocab_size):
    for j in range(vocab_size):
        count = co_matrix[i][j]
        if count > 0:
            co_occurrences.append(((index_to_word[i], index_to_word[j]), count))

# Sort and display top results
co_occurrences.sort(key=lambda x: x[1], reverse=True)
for pair, count in co_occurrences[:10]:
    print(f"{pair}: {count}")


Top co-occurring word pairs:
('</s>', '<s>'): 99
('.', '</s>'): 89
('<s>', 'The'): 27
(',', 'the'): 16
("''", '.'): 15
('in', 'the'): 15
('of', 'the'): 14
('<s>', '``'): 8
("''", ','): 7
('jury', 'said'): 7


With the code below you can verify if the co-occurence works

In [44]:
def check_co_occurrence(word1, word2, co_matrix, word_to_index):
    """
    Print the co-occurrence count for a specific word pair.
    """
    if word1 not in word_to_index or word2 not in word_to_index:
        print(f"One or both words '{word1}' and '{word2}' are not in the vocabulary.")
        return

    index1 = word_to_index[word1]
    index2 = word_to_index[word2]

    co_count = co_matrix[index1][index2]
    print(f"Co-occurrence count for '{word1}' and '{word2}': {co_count}")

check_co_occurrence('the', 'jury', co_matrix, word_to_index)
check_co_occurrence('the', 'cat', co_matrix, word_to_index)

Co-occurrence count for 'the' and 'jury': 7
One or both words 'the' and 'cat' are not in the vocabulary.


In this part, we reduce the dimensionality of the **co-occurrence matrix** using **Singular Value Decomposition (SVD)**. This technique helps in extracting the most important features from the matrix, making it easier to visualize and interpret the relationships between words.

Steps:
1. **Singular Value Decomposition (SVD)**: The co-occurrence matrix is decomposed into three matrices: \( M = U \cdot \Sigma \cdot V^T \).
2. **Dimensionality Reduction**: We retain only the top `k` components (default is 2) from the SVD, effectively reducing the dimensionality of the matrix.
3. **Reduced Matrix**: The reduced matrix retains the most important semantic information and can be used for tasks such as **word embedding visualization** or **semantic similarity analysis**.

Dimensionality reduction allows us to capture the essence of word relationships in fewer dimensions, making it more efficient to process and visualize the data.

In [16]:
def reduce_to_k_dim(M, k=2):
    raise NotImplementedError
    return M_reduced

In this final part, we visualize the reduced word embeddings in **2D space**. After reducing the dimensionality of the co-occurrence matrix using SVD, we plot the embeddings for each word to observe their relationships in a low-dimensional space.

Steps:
1. **Plotting the Embeddings**: Each word from the vocabulary is plotted based on its 2D coordinates from the reduced matrix.
2. **Visualization**: The scatter plot allows us to see how words cluster together or relate to each other based on co-occurrences in the text. Words with similar contexts should appear closer to each other in the plot.
3. **Interpretation**: By looking at the 2D plot, we can analyze word similarities, relationships, and clusters formed by words that often occur together.

This visualization is a useful tool for understanding the structure of word embeddings and the relationships between words in the corpus.







In [17]:
def plot_embeddings():
    raise NotImplementedError

words = ['movie', 'book', 'mysterious', 'story', 'fascinating', 'good', 'interesting', 'large', 'massive', 'huge']

plot_embeddings(M_reduced, vocab)

NameError: name 'M_reduced' is not defined

## Prediction-Based Word Vectors
In this section, we load pre-trained word vectors using the **GloVe** embeddings from the [Stanford NLP paper on GloVe](https://nlp.stanford.edu/pubs/glove.pdf). The pre-trained embeddings provide rich semantic information about words based on large text corpora, allowing us to represent words as vectors that capture their meaning and relationships.

Steps:
1. **Load GloVe Word Vectors**: We use the GloVe embeddings from the `glove-wiki-gigaword-200` model, which has 200-dimensional vectors for each word.
2. **Vocabulary Matching**: We map the words from our corpus to the GloVe vocabulary and extract the corresponding vectors.
3. **Dimensionality Reduction**: After obtaining the word vectors, we reduce the dimensionality to visualize the embeddings in 2D space, allowing us to explore word relationships visually.

This is an excellent opportunity to explore how pre-trained embeddings like GloVe can be used to improve downstream NLP tasks by providing richer word representations.

In [18]:
# Do not edit
import gensim.downloader as api
import numpy as np
wv_from_bin = api.load("glove-wiki-gigaword-200")
print("Loaded vocab size %i" % len(list(wv_from_bin.index_to_key)))

wv_words = list(wv_from_bin.index_to_key)
unique_tokens = set(tokens)
word2ind = {}
M = []
idx = 0
print('rendering M based on wv')
for w in wv_words:
    try:
        M.append(wv_from_bin.get_vector(w))
        word2ind[w] = idx
        idx += 1
    except:
        pass

print('rendering M based on unique_tokens')
for w in unique_tokens:
    if w in wv_words:
        continue
    try:
        M.append(wv_from_bin.get_vector(w))
        word2ind[w] = idx
        idx += 1
    except:
        pass
M = np.stack(M)
M_reduced = reduce_to_k_dim(M, k=2)

M_lengths = np.linalg.norm(M_reduced, axis=1)
M_reduced_normalized = M_reduced / M_lengths[:, np.newaxis]

ModuleNotFoundError: No module named 'gensim'

### Compare plots
What stands out when comparing the two different ways of embedding using the given words?

(Use this to help answer the question in your assignment document).

In [19]:
plot_embeddings(M_reduced_normalized, tokens)

NameError: name 'M_reduced_normalized' is not defined

### Words with multiple meanings
Polysemes and homonyms are words that have more than one meaning (see [this wiki page](https://en.wikipedia.org/wiki/Polysemy) to learn more about the difference between polysemes and homonyms). Your task is to find a word with at least two different meanings such that the top-10 most similar words (according to cosine similarity) contain related words from both meanings.

For example:
- "leaves" has both the "go_away" and "a_structure_of_a_plant" meanings in the top 10.
- "scoop" has both "handed_waffle_cone" and "lowdown" meanings in the top 10.

You will probably need to try several polysemous or homonymic words before you find one.

Once you discover a word that fits this criterion, state the word and explain the multiple meanings that occur in the top 10. Reflect on why many of the polysemous or homonymic words you tried didn’t work (i.e., why the top-10 most similar words only contain one of the word's meanings).

**Note**: Use the `wv_from_bin.most_similar(word)` function to get the top 10 most similar words. This function ranks all other words in the vocabulary based on their cosine similarity to the given word. For further assistance, you can refer to the [Gensim documentation](https://radimrehurek.com/gensim/models/keyedvectors.html).

In [20]:
## Your code here

### Synonyms
When considering **Cosine Similarity**, it's often useful to think of **Cosine Distance**, which is simply 1 - Cosine Similarity.

Find three words \( (w_1, w_2, w_3) \) where:
- \(w_1\) and \(w_2\) are **synonyms**.
- \(w_1\) and \(w_3\) are **antonyms**.

However, you need to find a case where **Cosine Distance** \( (w_1, w_3) < \text{Cosine Distance} (w_1, w_2) \). In other words, the antonym is **closer** to the word than its synonym in the vector space.

For example, "happy" (\(w_1\)) may be closer to "sad" (\(w_3\)) than to "cheerful" (\(w_2\)), which is counter-intuitive since we expect synonyms to be closer than antonyms. 

Once you find such an example, provide a possible explanation for why this result may have occurred.

Use the `wv_from_bin.distance(w1, w2)` function to compute the cosine distance between two words. You can refer to the [Gensim documentation](https://radimrehurek.com/gensim/models/keyedvectors.html) for further assistance.







In [21]:
## Your code here

### Analogies
Word vectors have demonstrated the ability to solve analogies based on their learned semantic relationships. For instance, consider the analogy: 

**"man : grandfather :: woman : x"** 

(Read: man is to grandfather as woman is to x). Using word vectors, we can find the word \(x\) that completes the analogy.

In the example provided, we use the `most_similar` function from the Gensim library. This function identifies words that are most similar to the words in the positive list and most dissimilar to those in the negative list. For analogy solving, it effectively computes:

\[ \text{word}(x) = \text{most similar to} (\text{woman} + \text{grandfather} - \text{man}) \]

The result is the word with the highest cosine similarity to the target vector. You can use this approach to explore various analogies and gain insights into the semantic structure of word vectors.

Refer to the [Gensim documentation](https://radimrehurek.com/gensim/models/keyedvectors.html) for more details on the `most_similar` function and how word vectors handle analogy-solving tasks.

In [22]:
# Run this cell to answer the analogy -- man : grandfather :: woman : x
print(wv_from_bin.most_similar(positive=['woman', 'grandfather'], negative=['man']))

NameError: name 'wv_from_bin' is not defined

### Bias in word vectors
a. **Observation**: Consider the results of a word vector model when queried with analogies involving "man", "woman", and "profession". What are the top results returned by the model for each query?

[Your Answer]

b. **Analysis**: Do you observe any gender biases in the results? For example, are certain professions more closely associated with "man" or "woman" based on the word vectors? Discuss how word embeddings might perpetuate societal biases present in the training data.

[Your Answer]

c. **Reflection**: How might such biases affect the fairness of machine learning models that use word embeddings? Suggest potential strategies for mitigating gender bias in word vector models.

[Your Answer]

In [23]:
print(wv_from_bin.most_similar(positive=['man', 'profession'], negative=['woman']))
print()
print(wv_from_bin.most_similar(positive=['woman', 'profession'], negative=['man']))

NameError: name 'wv_from_bin' is not defined